## Beam deflection with double integration

In [ ]:
%matplotlib widget
# Documentation needed for ipyml - notifications on push from github
from bmcs_utils.api import \
    InteractiveModel, InteractiveWindow, Item, View
import traits.api as tr
import matplotlib.pylab as plt
from matplotlib.path import Path
import matplotlib.patches as mpatches
from matplotlib.patches import PathPatch
import numpy as np
import sympy as sp
from sympy.functions import SingularityFunction, Piecewise, factorial
from scipy.integrate import cumtrapz
sp.init_printing()

In [ ]:
class BeamBC(tr.HasTraits):
    phi0_list = tr.List
    w0_list = tr.List
    
class BeamRC3Pt(BeamBC):
    pass

In [ ]:
x, M, L, P= sp.symbols('x, M, L, P')
PtoM = {P : M*4/L}
# M_ = 2*M/L - 4*M/L*(x-L/2)
M_ = sp.Piecewise(
    (0,x<0),
    (2*M*x/L, x <= L/2),
    ((2*M*x/L - (4*M/L)*(x-L/2)), x<=L),
    (0, True)
            )
M_
type(M_)


In [ ]:
from sympy.physics.continuum_mechanics.beam import Beam
from sympy import symbols
import sympy as sp
from sympy.functions import SingularityFunction, Piecewise, factorial

x, E, I, F = symbols('x E I F')
l = symbols('l', positive=True)
b = Beam(l, E, I)
R1,R2 = symbols('R1  R2')
b.apply_load(R1, 0, -1)
b.apply_load(R2, l, -1)
b.apply_load(-F, l/2, -1)
b.bc_deflection = [(0, 0),(l, 0)]
b.solve_for_reaction_loads(R1, R2)

In [ ]:
M_ = b.bending_moment().rewrite(Piecewise)
M_

In [ ]:
get_M = sp.lambdify((x,F,l), M_)

In [ ]:
# M_x = get_M(1,10,1)
# M_x

In [ ]:
phi_ = sp.integrate(M_,(x))
phi_

In [ ]:
phi_ = sp.integrate(M_,x)
# phi_ = sp.Piecewise((M*x**2/L - P*L**2/16, x <= L/2),
#     (P*x**2/4 - P*L**2/16 - P/2*(x-L/2)**2, x > L/2))
         
phi_

In [ ]:
# phi_ = phi_.subs(PtoM)
# phi_
type(phi_)

In [ ]:
get_phi = sp.lambdify((x,F,L), phi_)

In [ ]:
w_ = sp.integrate(phi_,x)
w_

In [ ]:
get_w = sp.lambdify((x,F,L), w_)

In [ ]:
from bmcs_beam.models.moment_curvature.moment_curvature_ import MomentCurvature
mc = MomentCurvature()

In [ ]:
class BeamDesign(InteractiveModel):
    
    name = 'BeamDesign'
   
    L = tr.Int(5000)
    H = tr.Int(200)
    ipw_view = View(
        Item('L', latex='L \mathrm{[mm]}', minmax=(1000,10000)),
        Item('H', latex='H \mathrm{[mm]}', minmax=(100,500))
    )

    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):  
        
        ax.fill([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='gray')
        ax.plot([0,self.L,self.L,0,0], [0,0,self.H,self.H,0],color='black')
        ax.annotate('L = {} mm'.format(np.round(self.L),0),
                xy=(self.L/2,self.H * 1.1), color='black')
        ax.axis('equal')
#         ax.autoscale(tight=True)

In [ ]:
InteractiveWindow(BeamDesign()).interact()

In [ ]:
class CrossSectionDesign(InteractiveModel):

    name = 'CrossSectionDesign'
   
    # Reinforcement
    E_carbon = tr.Int(200000)
    width = tr.Float(8)
    thickness = tr.Float(1)
    spacing = tr.Float(1)
    n_layers = tr.Int(1)
    A_roving = tr.Float(1)
    f_h = tr.Int(5)

    #Concerte cross section
#     L = tr.Int(5000, param=True, latex='L \mathrm{mm}', minmax=(10,10000))
    H = tr.Int(10)
    B = tr.Int(10)
    E_con = tr.Int(14000)
    F = tr.Float(10)
    n_x = tr.Int(100)
    
    ipw_view = View(
        Item('E_carbon', param=True, latex='E_r \mathrm{[MPa]}', minmax=(200000,300000)),
        Item('width', param=True, latex='rov_w \mathrm{[mm]}', minmax=(8,450)),
        Item('thickness', param=True, latex='rov_t \mathrm{[mm]}', minmax=(1,100)),
        Item('spacing', param=True, latex='ro_s \mathrm{[mm]}', minmax=(1,100)),
        Item('n_layers', param=True, latex='n_l \mathrm{[-]}', minmax=(1,100)),
        Item('A_roving', param=True, latex='A_r \mathrm{[mm^2]}', minmax=(1,100)),
        Item('f_h', param=True, latex='f_h \mathrm{[mm]}', minmax=(5,500)),
        Item('H', param=True, latex='H \mathrm{[mm]}', minmax=(10,500)),
        Item('B', param=True, latex='B \mathrm{[mm]}', minmax=(10,500)),
        Item('E_con', param=True, latex='E \mathrm{[MPa]}', minmax=(14000,41000)),
        Item('F', param=True, latex='F \mathrm{[N]}', minmax=(10,1000)),
        Item('n_x', param=True, latex='n_x \mathrm{[-]}', minmax=(1,1000))
    )
    
    def get_comp_E(self):
        '''todo: check it with the bmcs example'''
        A_composite = self.B * self.H
        n_rovings = self.width / self.spacing 
        A_layer = n_rovings * self.A_roving 
        A_carbon = self.n_layers * A_layer 
        A_concrete = A_composite - A_carbon 
        E_comp = (self.E_carbon * A_carbon + self.E_con * A_concrete) / (A_composite)
        return E_comp
      
    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):     
        ax.axis([0, self.B, 0, self.H]);
        ax.axis('equal');
        ax.fill([0,self.B,self.B,0,0], [0,0,self.H,self.H,0],color='gray')
        ax.plot([0,self.B,self.B,0,0], [0,0,self.H,self.H,0],color='black')
        ax.plot([self.B/2 - self.width/2, self.B/2 + self.width/2], 
                [self.f_h,self.f_h],color='Blue',
                linewidth=self.n_layers*self.thickness)       
        ax.annotate('E_composite = {} GPa'.format(np.round(self.get_comp_E()/1000),0),
                xy=(self.B/10,self.f_h*1.1), color='white')


In [ ]:
InteractiveWindow(CrossSectionDesign()).interact()

In [ ]:
class BoundaryConditions(InteractiveModel):
    
    name = 'BoundaryConditions'
    
    beam_design = tr.Instance(BeamDesign)
    def _beam_design_default(self):
        return BeamDesign()
    
    cross_section = tr.Instance(CrossSectionDesign)
    def _cross_section_default(self):
        return CrossSectionDesign()

    L = tr.Int(5000, param=True, latex='L \mathrm{[mm]}', minmax=(1000,10000))
    H = tr.Int(200, param=True, latex='H \mathrm{[mm]}', minmax=(10,500))
    F = tr.Float(1000)
    n_sup = tr.Int(2)
    G_adj = tr.Float(0.02)
    F_pos = tr.Int(2500)

    ipw_view = View(
        Item('F', param=True, latex='F \mathrm{[N]}', minmax=(1000,10000)),
        Item('n_sup', param=True, latex='n_{sup} \mathrm{[-]}', minmax=(2,10)), #number of supports
        Item('G_adj', param=True, latex='G_{adj} \mathrm{[-]}', minmax=(1e-3,1e-1)), #Graphic adjustment factor
        Item('F_pos', param=True, latex='F_{pos} \mathrm{[mm]}', minmax=(0,10000)), # Force position
    )

    def Support (self,n):
        
        slef.n = n
        pass 
    
    def subplots(self, fig):
        return fig.subplots(1, 1)

    def update_plot(self, ax):  
        
        ax.fill([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='gray')
        ax.plot([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='black')
                
        vertices = []
        codes = []

        codes = [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices = [(-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), (0, 0),
                    (self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), 
                    (-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj), (self.beam_design.L, 0),
                     (self.beam_design.L * (1 + self.G_adj), -self.beam_design.L * self.G_adj), 
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO] + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1+self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2)]

        vertices = np.array(vertices, float)
        path = Path(vertices, codes)

        pathpatch = PathPatch(path, facecolor='None', edgecolor='green')

        ax.add_patch(pathpatch)
        ax.plot((0,self.beam_design.L*1),(0,0))
#         ax.set_title('A compound path')
        # ax.set_size(3,3)
        # ax.autoscale_view()

        x_tail = self.F_pos
        y_tail = self.beam_design.L/20 + self.cross_section.H
        x_head = self.F_pos
        y_head = 0 + self.cross_section.H
        dy = y_head + x_head/10
        arrow = mpatches.FancyArrowPatch((x_tail, y_tail), (x_head, y_head),
                                         color = 'blue',mutation_scale=self.beam_design.L/500)
        ax.annotate('{} KN'.format(np.round(self.F/1000),0),
                xy=(x_tail,y_tail), color='black')
        ax.add_patch(arrow)
        ax.axis('equal')
        ax.autoscale(tight=True)

In [ ]:
InteractiveWindow(BoundaryConditions()).interact()

In [ ]:
class Beam3PtBending(InteractiveModel):
    '''
    Todo: 1- moving point load, line load, load combinations
    '''
    name = 'Analysis'

    mc = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_mc(self):
        mc_ = MomentCurvature()
        return mc_

    beam_design = tr.Instance(BeamDesign)
    def _beam_design_default(self):
        return BeamDesign()
    
    boundary_conditions = tr.Instance(BoundaryConditions)
    def _boundary_conditions_default(self):
        return BoundaryConditions()
    
    cross_section = tr.Instance(CrossSectionDesign)
    def _cross_section_default(self):
        return CrossSectionDesign()
    
    # Reinforcement
    E_carbon = tr.Int(200000)
    width = tr.Float(10)
    thickness = tr.Float(1)
    spacing = tr.Float(1)
    n_layers = tr.Int(1)
    A_roving = tr.Float(1)
    
    #Concerte cross section
    L = tr.Int(5000, param=True, latex='L \mathrm{mm}', minmax=(10,10000))
#     H = tr.Int(10, param=True, latex='H \mathrm{mm}', minmax=(10,500))
#     B = tr.Int(10, param=True, latex='B \mathrm{mm}', minmax=(10,500))
    E_con = tr.Int(14000)
    F = tr.Float(1000)
    n_x = tr.Int(100)
    G_adj = tr.Float(0.02)

    ipw_view = View(
        Item('E_con', param=True, latex='E \mathrm{MPa}', minmax=(14000,41000)) ,
        Item('F', param=True, latex='F \mathrm{N}', minmax=(10,100000)),
        Item('E_carbon', param=True, latex='E_r \mathrm{MPa}', minmax=(200000,300000)),
        Item('width', param=True, latex='rov_w \mathrm{mm}', minmax=(10,450)),
        Item('thickness', param=True, latex='rov_t \mathrm{mm}', minmax=(1,100)),
        Item('spacing', param=True, latex='ro_s \mathrm{mm}', minmax=(1,100)),
        Item('n_layers', param=True, latex='n_l \mathrm{-}', minmax=(1,100)),
        Item('A_roving', param=True, latex='A_r \mathrm{mm^2}', minmax=(1,100)),
        Item('G_adj', param=True, latex='G_{adj} \mathrm{-}', minmax=(1e-3,1e-1)),
        Item('n_x',param=True, latex='n_x \mathrm{-}', minmax=(1,1000))
    )

    x = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_x(self):
        return np.linspace(0,self.beam_design.L,self.n_x)
    
    E_comp = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_E_comp(self):
        A_composite = self.cross_section.B * self.cross_section.H
        n_rovings = self.cross_section.width / self.cross_section.spacing 
        A_layer = n_rovings * self.cross_section.A_roving 
        A_carbon = self.cross_section.n_layers * A_layer 
        A_concrete = A_composite - A_carbon 
        E_comp = (self.cross_section.E_carbon * A_carbon + self.cross_section.E_con * A_concrete) / (A_composite)
        return E_comp   

    def get_M_x(self):
#         M = self.F / 2 * self.beam_design.L / 2
        M_x = get_M(self.x, self.F, self.L)
        return M_x
    
    def get_kappa_x(self):
        M = self.get_M_x()
#         I = (self.B*self.H**3)/12
        return mc.get_kappa(M)
#         return M / I / self.E_comp
    
    def get_phi_x(self):
        kappa_x = self.get_kappa_x()
        phi_x = cumtrapz(kappa_x, self.x, initial=0)
        phi_L2 = np.interp(self.beam_design.L/2, self.x, phi_x)
        phi_x -= phi_L2
        return phi_x
    
    def get_w_x(self):
        phi_x = self.get_phi_x()
        w_x = cumtrapz(phi_x, self.x, initial=0)
        w_x += w_x[0]
        return w_x
    
    def subplots(self, fig):
        return fig.subplots(5, 1)

    def update_plot(self, axes):
        ax1, ax2, ax3, ax4, ax5 = axes
        
#         ax1.axis('equal');
        ax1.fill([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='gray')
        ax1.plot([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='black')
                
        vertices = []
        codes = []

        codes = [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices = [(-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), (0, 0),
                    (self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), 
                    (-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj), (self.beam_design.L, 0),
                     (self.beam_design.L * (1 + self.G_adj), -self.beam_design.L * self.G_adj), 
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO] + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1+self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2)]

        vertices = np.array(vertices, float)
        path = Path(vertices, codes)

        pathpatch = PathPatch(path, facecolor='None', edgecolor='green')

        ax1.add_patch(pathpatch)
        ax1.plot((0,self.beam_design.L*1),(0,0))
#         ax1.set_title('A compound path')
        # ax.set_size(3,3)
        # ax.autoscale_view()

        x_tail = self.beam_design.L/2
        y_tail = self.F * self.G_adj + self.cross_section.H
        x_head = self.beam_design.L/2
        y_head = 0 + self.cross_section.H
        dy = y_head + x_head/10
        arrow = mpatches.FancyArrowPatch((x_tail, y_tail), (x_head, y_head),
                                          mutation_scale=10)
        ax1.add_patch(arrow)
        ax1.axis('auto');
#         ax1.autoscale(tight=True)
        
        x = self.x
        
        M_x = self.get_M_x()
        ax2.plot(x, -M_x, color='red', label='moment [N.mm]')
        leg = ax2.legend();

        phi_x = self.get_phi_x()
        ax3.plot(x, phi_x, color='green', label='phi [-]')
        leg = ax3.legend();
        
        w_x = self.get_w_x()
        ax4.plot(x, w_x, color='blue', label='$w$ [mm]')
        leg = ax4.legend();
                
        kappa_x = self.get_kappa_x()
        ax5.plot(x, kappa_x, color='black', label='$kappa$ [-]')
        leg = ax5.legend();
        

In [ ]:
# bd = Beam3PtBending()
ibd = InteractiveWindow([BeamDesign(),BoundaryConditions(),CrossSectionDesign(),Beam3PtBending()])
ibd.interact()

In [ ]:
class Beam3PtBending(InteractiveModel):
    '''
    Todo: 1- moving point load, line load, load combinations
    '''
    name = 'Analysis'

    mc = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_mc(self):
        mc_ = MomentCurvature()
        return mc_

    beam_design = tr.Instance(BeamDesign)
    def _beam_design_default(self):
        return BeamDesign()
    
    boundary_conditions = tr.Instance(BoundaryConditions)
    def _boundary_conditions_default(self):
        return BoundaryConditions()
    
    cross_section = tr.Instance(CrossSectionDesign)
    def _cross_section_default(self):
        return CrossSectionDesign()
    
    # Reinforcement
    E_carbon = tr.Int(200000)
    width = tr.Float(10)
    thickness = tr.Float(1)
    spacing = tr.Float(1)
    n_layers = tr.Int(1)
    A_roving = tr.Float(1)
    
    #Concerte cross section
    L = tr.Int(5000, param=True, latex='L \mathrm{mm}', minmax=(10,10000))
#     H = tr.Int(10, param=True, latex='H \mathrm{mm}', minmax=(10,500))
#     B = tr.Int(10, param=True, latex='B \mathrm{mm}', minmax=(10,500))
    E_con = tr.Int(14000)
    F = tr.Float(1000)
    n_x = tr.Int(100)
    G_adj = tr.Float(0.02)

    ipw_view = View(
        Item('E_con', param=True, latex='E \mathrm{MPa}', minmax=(14000,41000)) ,
        Item('F', param=True, latex='F \mathrm{N}', minmax=(10,100000)),
        Item('E_carbon', param=True, latex='E_r \mathrm{MPa}', minmax=(200000,300000)),
        Item('width', param=True, latex='rov_w \mathrm{mm}', minmax=(10,450)),
        Item('thickness', param=True, latex='rov_t \mathrm{mm}', minmax=(1,100)),
        Item('spacing', param=True, latex='ro_s \mathrm{mm}', minmax=(1,100)),
        Item('n_layers', param=True, latex='n_l \mathrm{-}', minmax=(1,100)),
        Item('A_roving', param=True, latex='A_r \mathrm{mm^2}', minmax=(1,100)),
        Item('G_adj', param=True, latex='G_{adj} \mathrm{-}', minmax=(1e-3,1e-1)),
        Item('n_x',param=True, latex='n_x \mathrm{-}', minmax=(1,1000))
    )

    x = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_x(self):
        return np.linspace(0,self.beam_design.L,self.n_x)
    
    E_comp = tr.Property(depends_on = '+param')
    @tr.cached_property
    def _get_E_comp(self):
        A_composite = self.cross_section.B * self.cross_section.H
        n_rovings = self.cross_section.width / self.cross_section.spacing 
        A_layer = n_rovings * self.cross_section.A_roving 
        A_carbon = self.cross_section.n_layers * A_layer 
        A_concrete = A_composite - A_carbon 
        E_comp = (self.cross_section.E_carbon * A_carbon + self.cross_section.E_con * A_concrete) / (A_composite)
        return E_comp   

    def get_M_x(self):
#         M = self.F / 2 * self.beam_design.L / 2
        M_x = get_M(self.x, self.F, self.L)
        return M_x
    
    def get_kappa_x(self):
        M = self.get_M_x()
#         I = (self.B*self.H**3)/12
        return mc.get_kappa(M)
#         return M / I / self.E_comp
    
    def get_phi_x(self):
        kappa_x = self.get_kappa_x()
        phi_x = cumtrapz(kappa_x, self.x, initial=0)
        phi_L2 = np.interp(self.beam_design.L/2, self.x, phi_x)
        phi_x -= phi_L2
        return phi_x
    
    def get_w_x(self):
        phi_x = self.get_phi_x()
        w_x = cumtrapz(phi_x, self.x, initial=0)
        w_x += w_x[0]
        return w_x
    
    def subplots(self, fig):
        return fig.subplots(5, 1)

    def update_plot(self, axes):
        ax1, ax2, ax3, ax4, ax5 = axes
        
#         ax1.axis('equal');
        ax1.fill([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='gray')
        ax1.plot([0,self.beam_design.L,self.beam_design.L,0,0], [0,0,self.cross_section.H,self.cross_section.H,0],color='black')
                
        vertices = []
        codes = []

        codes = [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY]
        vertices = [(-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), (0, 0),
                    (self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj), 
                    (-self.beam_design.L * self.G_adj, -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO]*2 + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj), (self.beam_design.L, 0),
                     (self.beam_design.L * (1 + self.G_adj), -self.beam_design.L * self.G_adj), 
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj)]

        codes += [Path.MOVETO] + [Path.LINETO] + [Path.CLOSEPOLY] 
        vertices += [(self.beam_design.L * (1+self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2),
                     (self.beam_design.L * (1-self.G_adj), -self.beam_design.L * self.G_adj*1.2)]

        vertices = np.array(vertices, float)
        path = Path(vertices, codes)

        pathpatch = PathPatch(path, facecolor='None', edgecolor='green')

        ax1.add_patch(pathpatch)
        ax1.plot((0,self.beam_design.L*1),(0,0))
#         ax1.set_title('A compound path')
        # ax.set_size(3,3)
        # ax.autoscale_view()

        x_tail = self.beam_design.L/2
        y_tail = self.F * self.G_adj + self.cross_section.H
        x_head = self.beam_design.L/2
        y_head = 0 + self.cross_section.H
        dy = y_head + x_head/10
        arrow = mpatches.FancyArrowPatch((x_tail, y_tail), (x_head, y_head),
                                          mutation_scale=10)
        ax1.add_patch(arrow)
        ax1.axis('auto');
#         ax1.autoscale(tight=True)
        
        x = self.x
        
        M_x = self.get_M_x()
        ax2.plot(x, -M_x, color='red', label='moment [N.mm]')
        leg = ax2.legend();

        phi_x = self.get_phi_x()
        ax3.plot(x, phi_x, color='green', label='phi [-]')
        leg = ax3.legend();
        
        w_x = self.get_w_x()
        ax4.plot(x, w_x, color='blue', label='$w$ [mm]')
        leg = ax4.legend();
                
        kappa_x = self.get_kappa_x()
        ax5.plot(x, kappa_x, color='black', label='$kappa$ [-]')
        leg = ax5.legend();
        

In [ ]:
# bd = Beam3PtBending()
ibd = InteractiveWindow([BeamDesign(),BoundaryConditions(),CrossSectionDesign(),Beam3PtBending()])
ibd.interact()